# DataLoader

The goal of this notebook is to load the contents of the CSV files provided by the movielens project into content within the database.

There will be a corresponding exporter notebook used to enable Personalize

In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import django
django.setup()

from movielens.models import User
from movielens.models import Item
from movielens.models import UserData
from movielens.models import Poster

from django.conf import settings
print(settings.DATABASES)

{'default': {'ENGINE': 'django.db.backends.sqlite3', 'NAME': 'local_copy.sqlite', 'ATOMIC_REQUESTS': False, 'AUTOCOMMIT': True, 'CONN_MAX_AGE': 0, 'OPTIONS': {}, 'TIME_ZONE': None, 'USER': '', 'PASSWORD': '', 'HOST': '', 'PORT': '', 'TEST': {'CHARSET': None, 'COLLATION': None, 'NAME': None, 'MIRROR': None}}}


In [2]:
# Start with the user

data = pd.read_csv('./data/ml-100k/u.user', sep='|', names=['USER_ID', 'AGE', 'GENDER', 'OCCUPATION', 'ZIP_CODE'])
pd.set_option('display.max_rows', 5)
data.head()

,USER_ID,AGE,GENDER,OCCUPATION,ZIP_CODE
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [3]:
for index, row in data.iterrows():
    user = User()
    user.user_id = row['USER_ID']
    user.age = row['AGE']
    user.gender = row['GENDER']
    user.occupation = row['OCCUPATION']
    user.zip_code = row['ZIP_CODE']
    user.save()

In [4]:
# Now to do the Item data

headers = ['MOVIE_ID', 'MOVIE_TITLE', 'RELEASE_DATE', 'VIDEO_RELEASE_DATE', 'IMDB_URL',
          'UNKNOWN', 'ACTION', 'ADVENTURE', 'ANIMATION', 'CHILDREN', 'COMEDY', 'CRIME',
          'DOCUMENTARY', 'DRAMA', 'FANTASY', 'FILM_NOIR', 'HORROR', 'MUSICAL', 'MYSTERY',
          'ROMANCE', 'SCI_FI', 'THRILLER', 'WAR', 'WESTERN']
item_data = pd.read_csv('./data/ml-100k/u.item', sep='|', names=headers, encoding="iso-8859-1", na_filter=False)
pd.set_option('display.max_rows', 5)
item_data.head()

,MOVIE_ID,MOVIE_TITLE,RELEASE_DATE,VIDEO_RELEASE_DATE,IMDB_URL,UNKNOWN,ACTION,ADVENTURE,ANIMATION,CHILDREN,...,FANTASY,FILM_NOIR,HORROR,MUSICAL,MYSTERY,ROMANCE,SCI_FI,THRILLER,WAR,WESTERN
0,1,Toy Story (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# Now we need to create the items


In [6]:
headers = ['MOVIE_ID', 'MOVIE_TITLE', 'RELEASE_DATE', 'VIDEO_RELEASE_DATE', 'IMDB_URL',
          'UNKNOWN', 'ACTION', 'ADVENTURE', 'ANIMATION', 'CHILDREN', 'COMEDY', 'CRIME',
          'DOCUMENTARY', 'DRAMA', 'FANTASY', 'FILM_NOIR', 'HORROR', 'MUSICAL', 'MYSTERY',
          'ROMANCE', 'SCI_FI', 'THRILLER', 'WAR', 'WESTERN']
for index, row in item_data.iterrows():
    item = Item()
    item.movie_id = row['MOVIE_ID']
    item.movie_title = row['MOVIE_TITLE']
    item.release_date = row['RELEASE_DATE']
    item.video_release_date = row['VIDEO_RELEASE_DATE']
    item.IMDB_URL = row['IMDB_URL']
    item.unknown = row['UNKNOWN']
    item.Action = row['ACTION']
    item.Adventure = row['ADVENTURE']
    item.Animation = row['ANIMATION']
    item.Children = row['CHILDREN']
    item.Comedy = row['COMEDY']
    item.Crime = row['CRIME']
    item.Documentary = row['DOCUMENTARY']
    item.Drama = row['DRAMA']
    item.Fantasy = row['FANTASY']
    item.Film_Noir = row['FILM_NOIR']
    item.Horror = row['HORROR']
    item.Musical = row['MUSICAL']
    item.Mystery = row['MYSTERY']
    item.Romance = row['ROMANCE']
    item.Sci_Fi = row['SCI_FI']
    item.Thriller = row['THRILLER']
    item.War = row['WAR']
    item.Western = row['WESTERN']
    item.save()

In [7]:
# Now for the Iteractions with Users and Items

headers = ['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP']
user_data = pd.read_csv('./data/ml-100k/u.data', sep='\t', names=headers)
pd.set_option('display.max_rows', 5)
user_data.head()

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [8]:
for index, row in user_data.iterrows():
    user_data = UserData()
    user_data.user_id = User.objects.get(pk=row['USER_ID'])
    user_data.item_id = Item.objects.get(pk=row['ITEM_ID'])
    user_data.rating = row['RATING']
    user_data.timestamp = row['TIMESTAMP']
    user_data.save()

In [9]:
# Configure Posters
posters = Poster.objects.all()
for poster in posters:
    posters.delete()
items = Item.objects.all()
for item in items:
    poster = Poster()
    poster.item = item
    poster.filename = str(item.movie_id) + ".jpg"
    poster.save()

In [10]:
# Fix URLS for items
# It turns out the dataset is a bit busted so lets clean that up

urldf = pd.read_csv('./data/ml-100k/movie_url.csv', names=['movie_id', 'url'])

In [11]:
for index, row in urldf.iterrows():
    item = Item.objects.get(movie_id=row['movie_id'])
    item.IMDB_URL = row['url']
    item.save()

Now all of your items are created in the local database, this will allow you to interact with them in the web interface. Your next step is to move onto getting the data into Personalize, go back to the docs and click the next link. Feel free to close this tab.